# Machine learning for solid solutions

## progress


## Atomate and Automatminer

Note: Atomate is primarily built to work with the VASP electronic structure software, but it is the intention of atomate to support a variety of software <br>
https://atomate.org   <br>
https://materialsproject.org